In [50]:
!pip install yfinance


In [55]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json

from pandas_datareader import data as pdr
import yfinance as yf
import datetime as dt


In [56]:
producer = KafkaProducer(bootstrap_servers=['1'], # Change Accordingly
                         value_serializer=lambda x:
                         dumps(x).encode('utf-8'))

In [57]:
yf.pdr_override()
# Load Data
company = "QYLD"
start = dt.datetime(2014,1,1)
end = dt.datetime.now()
df = pdr.get_data_yahoo(company, start, end)

# df = pd.read_csv("data/indexProcessed.csv")
df = df.reset_index()
df['Date'] = df['Date'].astype(str)

df.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-01-02,25.440001,25.440001,25.440001,25.440001,10.044307,300
1,2014-01-03,25.420000,25.420000,25.420000,25.420000,10.036413,300
2,2014-01-06,25.389999,25.389999,25.389999,25.389999,10.024569,600
3,2014-01-07,25.520000,25.520000,25.520000,25.520000,10.075896,100
4,2014-01-08,25.580000,25.580000,25.570000,25.570000,10.095639,1700


In [58]:
for i in range(0, len(df)):
    dict_stock = df.to_dict(orient="records")[i]
    producer.send('demo_test', value=dict_stock)

In [8]:
while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_test', value=dict_stock)
    sleep(1)

KeyboardInterrupt: 

In [25]:
producer.flush()